<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
<b>Importando librerías necesarias para el proyecto<B>
</div>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.io import arff
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import auc, classification_report, precision_recall_curve, average_precision_score, accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# import graphviz
#import pydotplus

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
    <b>Cargando y mostrando el Dataset Dry Bean</b>
</div>

In [2]:
data = arff.loadarff('Dry_Bean_Dataset.arff')
df= pd.DataFrame(data[0])
df.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395.0,610.291,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,b'SEKER'
1,28734.0,638.018,200.524796,182.734419,1.097356,0.411785,29172.0,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,b'SEKER'
2,29380.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,b'SEKER'
3,30008.0,645.884,210.557999,182.516516,1.153638,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,b'SEKER'
4,30140.0,620.134,201.847882,190.279279,1.060798,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,b'SEKER'


<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
    <b>Viendo las informaciones de las columnas</b>
</div>

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13611 entries, 0 to 13610
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Area             13611 non-null  float64
 1   Perimeter        13611 non-null  float64
 2   MajorAxisLength  13611 non-null  float64
 3   MinorAxisLength  13611 non-null  float64
 4   AspectRation     13611 non-null  float64
 5   Eccentricity     13611 non-null  float64
 6   ConvexArea       13611 non-null  float64
 7   EquivDiameter    13611 non-null  float64
 8   Extent           13611 non-null  float64
 9   Solidity         13611 non-null  float64
 10  roundness        13611 non-null  float64
 11  Compactness      13611 non-null  float64
 12  ShapeFactor1     13611 non-null  float64
 13  ShapeFactor2     13611 non-null  float64
 14  ShapeFactor3     13611 non-null  float64
 15  ShapeFactor4     13611 non-null  float64
 16  Class            13611 non-null  object 
dtypes: float64(1

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
    <b>Codificando la cadena de clase a UTF-8 para eliminar el carácter adicional que aparece al cargarlo del ARFF</b>
</div>

In [4]:
df[["Class"]]=df[["Class"]].apply(lambda x: x.str.decode('utf8'))
df.sample(10)

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
10455,26272.0,667.302,219.510657,153.016747,1.434553,0.716992,26984.0,182.894913,0.699505,0.973614,0.741409,0.833194,0.008355,0.002484,0.694212,0.995884,DERMASON
11698,31408.0,648.687,235.231123,170.716953,1.377901,0.687968,31740.0,199.974767,0.811555,0.989540,0.937950,0.850120,0.007490,0.002413,0.722705,0.995816,DERMASON
3482,157119.0,1483.123,530.002228,379.431894,1.396831,0.698196,158406.0,447.269632,0.773871,0.991875,0.897603,0.843901,0.003373,0.001055,0.712170,0.994781,BOMBAY
2799,71536.0,1020.470,341.898522,267.109084,1.279996,0.624215,72443.0,301.798714,0.790261,0.987480,0.863245,0.882714,0.004779,0.001790,0.779185,0.997353,BARBUNYA
2925,73944.0,1067.612,369.354891,255.971557,1.442953,0.720915,75339.0,306.836153,0.718796,0.981484,0.815241,0.830735,0.004995,0.001467,0.690121,0.995812,BARBUNYA
6452,53684.0,902.240,364.958131,189.896292,1.921881,0.853969,54390.0,261.443286,0.663749,0.987020,0.828725,0.716365,0.006798,0.001104,0.513179,0.986269,HOROZ
6017,49417.0,889.713,369.567234,171.401610,2.156148,0.885945,49965.0,250.837953,0.716937,0.989032,0.784488,0.678734,0.007479,0.000979,0.460680,0.993295,HOROZ
4849,76890.0,1076.649,416.749392,237.324692,1.756030,0.822015,78603.0,312.888780,0.792337,0.978207,0.833550,0.750784,0.005420,0.001062,0.563677,0.989833,CALI
5698,44074.0,841.284,338.663855,167.218120,2.025282,0.869599,44765.0,236.889763,0.634195,0.984564,0.782541,0.699483,0.007684,0.001135,0.489277,0.990924,HOROZ
10031,55427.0,879.739,332.046268,213.277395,1.556875,0.766443,55953.0,265.653625,0.814911,0.990599,0.899961,0.800050,0.005991,0.001514,0.640080,0.996525,SIRA


<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
    <b>Separando los atributos no clase de la clase</b>
</div>

In [5]:
X = df.drop(["Class"], axis=1)
y = df["Class"]

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
    <b>Escalamiento mínimo-máximo, usaremos aparte el dataset en la variable escalado1</b>
</div>

In [6]:
escalado1 = df.copy()

num_attrs = ['Area','Perimeter','MajorAxisLength','MinorAxisLength','AspectRation','Eccentricity','ConvexArea','EquivDiameter',
            'Extent','Solidity','roundness','Compactness','ShapeFactor1','ShapeFactor3','ShapeFactor4']
scaler = MinMaxScaler()
escalado1[num_attrs] = scaler.fit_transform(escalado1[num_attrs])

escalado1[num_attrs].describe()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor3,ShapeFactor4
count,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000
mean,0.139320,0.226304,0.245905,0.236190,0.397295,0.768181,0.136386,0.224978,0.625380,0.900112,0.765693,0.459403,0.493356,0.413253,0.910285
std,0.125212,0.146710,0.154332,0.133171,0.175517,0.132860,0.122744,0.144996,0.157895,0.061783,0.118786,0.177989,0.147006,0.175392,0.083898
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.067926,0.122404,0.125531,0.157944,0.289902,0.717685,0.066084,0.131880,0.525344,0.880592,0.683497,0.351551,0.406861,0.302997,0.884146
50%,0.103469,0.184992,0.204017,0.207054,0.374443,0.787743,0.100974,0.189141,0.657953,0.915228,0.785402,0.463478,0.503986,0.410514,0.935698
75%,0.174691,0.309781,0.347394,0.279902,0.485429,0.854208,0.171533,0.289620,0.744779,0.938164,0.852681,0.558634,0.585601,0.506118,0.964454
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
escalado1.sample(15)

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
5132,0.262460,0.395122,0.423318,0.383564,0.452677,0.836901,0.257469,0.396079,0.705610,0.870764,0.714353,0.376948,0.304249,0.001116,0.326925,0.776792,CALI
3498,0.592781,0.682680,0.659889,0.732222,0.329119,0.752846,0.579882,0.708216,0.723266,0.897771,0.747190,0.513702,0.088074,0.000957,0.460494,0.946583,BOMBAY
4615,0.228505,0.358514,0.385119,0.346003,0.452255,0.836668,0.224964,0.356684,0.625652,0.838027,0.709849,0.378983,0.338524,0.001178,0.328855,0.804770,CALI
10921,0.033984,0.072456,0.106559,0.078572,0.429730,0.823720,0.033033,0.070673,0.737119,0.923655,0.812825,0.410760,0.752821,0.001983,0.359208,0.978449,DERMASON
9601,0.122393,0.205526,0.206588,0.259177,0.281363,0.709308,0.119356,0.217449,0.596038,0.922865,0.831753,0.569456,0.430012,0.001849,0.517236,0.956229,SIRA
2891,0.225388,0.383316,0.339209,0.381405,0.323878,0.748438,0.221385,0.352964,0.666743,0.859907,0.583474,0.519803,0.300128,0.001423,0.466639,0.950367,BARBUNYA
2708,0.211485,0.406989,0.331189,0.356804,0.346841,0.767152,0.208923,0.336139,0.740269,0.807700,0.423344,0.494599,0.322654,0.001409,0.441357,0.953485,BARBUNYA
13605,0.092444,0.161550,0.167652,0.211536,0.285875,0.713769,0.089761,0.171998,0.564522,0.949412,0.846150,0.564951,0.495092,0.001986,0.512602,0.970095,DERMASON
7491,0.066615,0.140145,0.177446,0.120713,0.500248,0.861510,0.065060,0.129649,0.817564,0.908783,0.720683,0.341759,0.658707,0.001604,0.293845,0.921053,SIRA
12821,0.066820,0.120410,0.135094,0.164756,0.303682,0.730634,0.065068,0.129998,0.505576,0.925514,0.865717,0.542427,0.572378,0.002085,0.489562,0.943816,DERMASON


<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
    <b>Escalamiento mínimo-máximo, usaremos aparte el dataset en la variable escalado2</b>
</div>

In [10]:
escalado2 = df.copy()

num_attrs = ['Area','Perimeter','MajorAxisLength','MinorAxisLength','AspectRation','Eccentricity','ConvexArea','EquivDiameter',
            'Extent','Solidity','roundness','Compactness','ShapeFactor1','ShapeFactor3','ShapeFactor4']

scaler = StandardScaler()
escalado2[num_attrs] = scaler.fit_transform(escalado2[num_attrs])

escalado2[num_attrs].describe()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor3,ShapeFactor4
count,1.361100e+04,1.361100e+04,1.361100e+04,1.361100e+04,1.361100e+04,1.361100e+04,1.361100e+04,1.361100e+04,1.361100e+04,1.361100e+04,1.361100e+04,1.361100e+04,1.361100e+04,1.361100e+04,1.361100e+04
mean,3.341028e-17,-1.002308e-16,-1.336411e-16,3.675131e-16,-4.343337e-16,-2.276075e-16,1.002308e-16,-1.336411e-16,-1.992088e-15,-2.298627e-14,3.341028e-17,-1.670514e-17,-1.670514e-16,7.517313e-17,1.966195e-14
std,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00
min,-1.112719e+00,-1.542583e+00,-1.593407e+00,-1.773645e+00,-2.263655e+00,-5.782098e+00,-1.111184e+00,-1.551678e+00,-3.960880e+00,-1.456945e+01,-6.446210e+00,-2.581167e+00,-3.356153e+00,-2.356255e+00,-1.085036e+01
25%,-5.702102e-01,-7.082260e-01,-7.799910e-01,-5.875797e-01,-6.118921e-01,-3.800848e-01,-5.727750e-01,-6.420997e-01,-6.335853e-01,-3.159613e-01,-6.919874e-01,-6.059668e-01,-5.884012e-01,-6.286495e-01,-3.115721e-01
50%,-2.863377e-01,-2.816033e-01,-2.714228e-01,-2.187975e-01,-1.302079e-01,1.472404e-01,-2.885152e-01,-2.471687e-01,2.063003e-01,2.446575e-01,1.659247e-01,2.289701e-02,7.231301e-02,-1.561590e-02,3.029170e-01
75%,2.824987e-01,5.690149e-01,6.576317e-01,3.282530e-01,5.021578e-01,6.475228e-01,2.863522e-01,4.458347e-01,7.562196e-01,6.159095e-01,7.323368e-01,5.575363e-01,6.275166e-01,5.294956e-01,6.456801e-01
max,6.874044e+00,5.273833e+00,4.886372e+00,5.735751e+00,3.434007e+00,1.744901e+00,7.036141e+00,5.345339e+00,2.372678e+00,1.616807e+00,1.972582e+00,3.037363e+00,3.446551e+00,3.345475e+00,1.069377e+00


In [13]:
escalado2.sample(15)

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
4360,0.589970,0.696791,0.826772,0.624794,0.462170,0.620658,0.575062,0.781040,1.506125,1.118300,0.044898,-0.557664,-0.891599,0.001177,-0.582641,-0.143191,CALI
3857,5.839055,4.271844,3.737860,5.494275,-0.640306,-0.416654,5.789106,4.753709,0.617839,0.989175,0.430169,0.559115,-3.287264,0.000854,0.531138,-0.644760,BOMBAY
4010,0.364789,0.414667,0.426246,0.573108,-0.077850,0.195699,0.352988,0.537835,0.791071,0.979581,0.426004,-0.017929,-0.858533,0.001405,-0.056338,0.644487,CALI
8814,-0.278528,-0.262712,-0.269129,-0.206493,-0.177761,0.101913,-0.279951,-0.236849,0.923976,0.151795,0.171144,0.077738,0.049357,0.001712,0.039287,0.380730,SIRA
3120,0.891167,1.182721,0.707239,1.446006,-0.643789,-0.421185,0.913111,1.089139,-0.921941,-1.945966,-1.072724,0.551966,-1.555908,0.001435,0.523703,-1.021858,BARBUNYA
11981,-0.701268,-0.842663,-0.836290,-0.769426,-0.410650,-0.140037,-0.702973,-0.839709,-0.120739,0.452215,0.393589,0.301613,0.962306,0.002117,0.265815,-0.527014,DERMASON
2940,0.729861,0.978994,0.691440,1.078326,-0.284920,-0.005108,0.725933,0.926405,0.970070,0.105888,-0.815392,0.188882,-1.301182,0.001363,0.151266,0.302046,BARBUNYA
11268,-0.791572,-1.011124,-0.971527,-0.922597,-0.445475,-0.179348,-0.791978,-0.982762,0.072051,0.263941,0.774090,0.371373,1.219805,0.002244,0.337190,0.539233,DERMASON
12483,-0.630402,-0.736299,-0.611036,-0.834454,0.170967,0.407121,-0.631904,-0.731489,0.612883,0.358573,0.326859,-0.266885,1.049709,0.001800,-0.301883,0.564120,DERMASON
11299,-0.788469,-0.935765,-0.936638,-0.943377,-0.334734,-0.057299,-0.788082,-0.977743,0.562659,0.096466,0.066399,0.225268,1.286966,0.002168,0.188132,-0.296949,DERMASON


<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
    <b>Escalamiento mínimo-máximo, usaremos aparte el dataset en la variable escalado3</b>
</div>

In [14]:
escalado3 = df.copy()

num_attrs = ['Area','Perimeter','MajorAxisLength','MinorAxisLength','AspectRation','Eccentricity','ConvexArea','EquivDiameter',
            'Extent','Solidity','roundness','Compactness','ShapeFactor1','ShapeFactor3','ShapeFactor4']

scaler = RobustScaler()
escalado3[num_attrs] = scaler.fit_transform(escalado3[num_attrs])

escalado3[num_attrs].describe()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor3,ShapeFactor4
count,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000
mean,0.335798,0.220478,0.188800,0.238905,0.116878,-0.143285,0.335824,0.227191,-0.148438,-0.262544,-0.116494,-0.019679,-0.059472,0.013484,-0.316444
std,1.172776,0.782966,0.695618,1.091942,0.897659,0.973170,1.164015,0.919207,0.719552,1.073150,0.702113,0.859505,0.822454,0.863481,1.044695
min,-0.969125,-0.987268,-0.919563,-1.697742,-1.915037,-5.770041,-0.957564,-1.199070,-2.998392,-15.897174,-4.642296,-2.238124,-2.819653,-2.021024,-11.651350
25%,-0.332907,-0.334019,-0.353756,-0.402674,-0.432372,-0.513158,-0.330870,-0.363010,-0.604319,-0.601606,-0.602329,-0.540492,-0.543387,-0.529324,-0.641930
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.667093,0.665981,0.646244,0.597326,0.567628,0.486842,0.669130,0.636990,0.395681,0.398394,0.397671,0.459508,0.456613,0.470676,0.358070
max,8.397216,4.349560,3.587725,6.501786,3.199332,1.554738,8.525694,5.140482,1.558764,1.472467,1.268431,2.590853,2.775054,2.902133,0.800688


In [17]:
escalado3.sample(15)

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
2075,0.258159,0.335095,0.166221,0.335510,-0.045127,-0.046672,0.265877,0.258674,-0.644616,-1.164941,-0.781279,0.035004,-0.317499,0.001599,0.035186,-0.538455,BARBUNYA
7185,0.680931,0.733861,0.857381,0.108210,1.794786,1.139804,0.673547,0.649217,-2.225902,0.008202,-1.197590,-1.463115,-0.086956,0.000946,-1.372181,-1.302012,HOROZ
7115,0.643137,0.764257,0.910997,-0.065649,2.202222,1.285419,0.638949,0.615741,-2.226905,-0.255066,-1.488677,-1.693030,0.066378,0.000887,-1.570303,-0.035545,HOROZ
13142,-0.268637,-0.233041,-0.221182,-0.309656,-0.183665,-0.199134,-0.262867,-0.289921,-0.550627,-0.409800,0.100926,0.192179,0.337337,0.001935,0.194540,0.136164,DERMASON
4142,0.894297,0.657526,0.491313,1.158779,-0.228810,-0.252044,0.887899,0.833571,-0.056876,-0.276545,0.033917,0.222842,-0.956432,0.001468,0.225887,-0.556096,CALI
4430,1.064750,0.824975,0.823438,0.900608,0.670759,0.559076,1.049239,0.975631,0.234285,0.325575,-0.277173,-0.626568,-0.770415,0.001128,-0.611196,-0.898930,CALI
12892,-0.328347,-0.336739,-0.277421,-0.378360,-0.238855,-0.264047,-0.329913,-0.357772,-0.465164,0.836562,0.519568,0.259975,0.411042,0.002010,0.263961,0.554189,DERMASON
327,-0.373940,-0.423564,-0.494753,-0.041065,-1.143319,-1.860346,-0.372720,-0.410523,0.319797,0.488662,0.931987,1.355387,0.027803,0.002688,1.442931,0.576020,SEKER
13375,-0.204487,-0.197658,-0.155100,-0.257866,-0.084792,-0.088869,-0.203835,-0.218496,-0.376697,0.249252,0.260943,0.090044,0.277215,0.001845,0.090736,0.104659,DERMASON
4361,1.027716,0.839985,0.824557,0.840403,0.741935,0.606571,1.016947,0.945135,1.004956,-0.025407,-0.489002,-0.698834,-0.715350,0.001112,-0.679418,-1.506527,CALI
